### Input Libraries

In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install tensorflow

In [ ]:
!pip install git+https://github.com/tensorflow/docs

In [ ]:
!pip install ipython

In [ ]:
# Required to save models in HDF5 format
!pip install pyyaml h5py  

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
import pathlib
import shutil
import tempfile
from IPython import display

In [ ]:
print(os.getcwd())
# Change current directoru to dataset folder structure directory
os.chdir('G:\ML\Load_Data_Category_180')
print(os.getcwd())
# class names
class_names=os.listdir() 
# no of classes
num_class= len(class_names)
print(class_names)
print(num_class)

In [ ]:
img_height=180
img_width=180
batch_size=32
epochs=1000

# Build Input Pipeline
## Tensorflow Training dataset

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.getcwd(),
    labels='inferred',
    label_mode='int',
    class_names= class_names,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_width,img_width),
    shuffle=True,
    seed=100,
    validation_split=0.2,
    subset="training",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
train_ds

In [ ]:
for ima, lab in train_ds.take(1):
    for x in lab:
        print(x.numpy())

## Visualize training data sample

In [ ]:
plt.figure(figsize=(10,10))
for image, label in train_ds.take(1):
    for x in range(9):
        ax=plt.subplot(3, 3, x+1)
        plt.imshow(image[x].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[label[x].numpy()])
        plt.axis("off")
plt.show()

## Tensorflow Validation dataset

In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    os.getcwd(),
    labels='inferred',
    label_mode='int',
    class_names=class_names,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height,img_width),
    shuffle=True,
    seed=100,
    validation_split=0.2,
    subset="validation",
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
validation_ds

## Visualize validation data sample

In [ ]:
plt.figure(figsize=(10,10))
for image, label in validation_ds.take(1):
    for x in range(9):
        ax=plt.subplot(3, 3, x+1)
        plt.imshow(image[x].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[label[x].numpy()])
        plt.axis("off")
plt.show()

## Configure the dataset for performance

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Data Augumentation

In [ ]:
data_augmentation = tf.keras.Sequential([
            tf.keras.layers.RandomFlip(input_shape=(img_height, img_width, 1)),
            tf.keras.layers.RandomRotation(0.1),
            tf.keras.layers.RandomZoom(0.1),
])

## Visualize data augmented sample

In [ ]:
plt.figure(figsize=(10,10))
for image, label in train_ds.take(1):
    for x in range(9):
        augemented_images=data_augmentation(image, training=True)
        ax=plt.subplot(3, 3, x+1)
        plt.imshow(augemented_images[1].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[label[1].numpy()])
        plt.axis("off")
plt.show()

In [ ]:

augemented_train_ds=train_ds.map(
lambda x,y: (data_augmentation(x, training=True),y))


### Learning rate scheduler

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

### Early stopping and learning rate callbacks

In [ ]:

def get_callbacks():
    return [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    tf.keras.callbacks.LearningRateScheduler(scheduler)
    ]


# Build the Model
## Set up the Model Layers

In [ ]:
model = tf.keras.Sequential([
          data_augmentation,
          tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
          tf.keras.layers.Conv2D(32, 3, activation='relu'),
          tf.keras.layers.Conv2D(32, 3, activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(32, 3, activation='relu'),
          tf.keras.layers.Conv2D(32, 3, activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Conv2D(32, 3, activation='relu'),
          tf.keras.layers.Conv2D(32, 3, activation='relu'),
          tf.keras.layers.MaxPooling2D(),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(128, kernel_regularizer=regularizers.l2(0.0001), activation='relu'),
          tf.keras.layers.Dense(num_class, activation='softmax')
          ])

In [ ]:
# Display the model's architecture

model.summary()

In [ ]:
model.compile(optimizer='Adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:

history = model.fit(train_ds, validation_data = validation_ds, epochs=epochs, callbacks=get_callbacks())

## Visualize the training results

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss'] 
val_loss = history.history['val_loss']
#epochs_range = range(epochs)
epochs_range = range(len(val_acc))
plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, train_acc, label= 'Training Accuracy')
plt.plot(epochs_range, val_acc, label= 'Validation Accuracy')
#loc='lower right'
plt.legend()
plt.title('Training and validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
#loc='Upper right'
plt.legend()
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
best_epoch= val_acc.index(max(val_acc))+1
print('Best epoch value',best_epoch)
best_train_accuracy=max(train_acc)*100
print('Maxium train accuracy',best_train_accuracy)
best_val_accuracy=max(val_acc)*100
print('Maxium validation accuracy',best_val_accuracy)

### Save the model

In [ ]:
#Manually save weights
# Save the weights
# Change directory
model.save_weights('D:\ModelOutputs\checkpoints\FD_Good_01')


In [ ]:
#Save model
# Change directory
model.save('D:\ModelOutputs\saved_model\FD_Good_01')


In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(validation_ds.take(1))


### Verify and plot the predictions

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.

num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for image, label in validation_ds.take(1):
    for x in range(num_images):
            predicted_label = np.argmax(predictions[x])
            true_label=label[x].numpy()
            
            if predicted_label == true_label:
                color = 'blue'
            else:
                color = 'red'
                
            plt.subplot(num_rows, 2*num_cols, 2*x+1)
            plt.grid(False)
            plt.xticks([])
            plt.yticks([])

            plt.imshow(image[x].numpy().astype("uint8"), cmap='gray')
            plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions[x]),
                                class_names[true_label]),
                                color=color)
            plt.subplot(num_rows, 2*num_cols, 2*x+2)
            plt.grid(False)
            plt.xticks(range(num_class))
            plt.yticks([])
            #plt.xlabel(label[x].numpy())
            thisplot = plt.bar(range(num_class), predictions[x], color="#777777")
            plt.ylim([0, 1])
            thisplot[predicted_label].set_color('red')
            thisplot[true_label].set_color('blue')
            #plt.axis("off")
plt.tight_layout()
plt.show()
